# 라이브러리 임포트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
!pip install finance-datareader
import FinanceDataReader as fdr
from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import ElasticNetCV
import os 
from tqdm import tqdm
import warnings 
warnings.filterwarnings(action = 'ignore') 
pd.options.display.max_columns = None

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 데이터셋 로드

In [ ]:
path = '/content/drive/MyDrive/stock_2/'

stock_list = pd.read_csv(path + 'stock_list.csv')
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x: str(x).zfill(6))
stock_codes = np.sort(stock_list['종목코드'].values)

In [ ]:
# start_date = '20210104'
# end_date = '20211105'

# start_weekday = pd.to_datetime(start_date).weekday()
# max_weeknum = pd.to_datetime(end_date).strftime('%V')
# Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

# print(f'WEEKDAY of "start_date" : {start_weekday}')
# print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
# print(f'HOW MANY "Business_days" : {Business_days.shape}', )
# display(Business_days.head())

# 전처리를 위한 함수 정의

In [ ]:
# 이동평균 추가하는 함수 정의 
def add_rolling_mean(df:pd.DataFrame, col_name:str, window_list:list)->pd.DataFrame:
  for window in window_list:
    df[col_name + f'_rolling{window}'] = df[col_name].rolling(window).mean().fillna(method = 'bfill') 
  return df

# 컬럼값을 lag하는 함수 정의
def lag_features(df:pd.DataFrame, col_list:list, lag_num:int)->pd.DataFrame:
  for col in col_list:
    df[col] = df[col].shift(lag_num).fillna(method = 'ffill').fillna(method = 'bfill')
  return df

# 종목별 전처리

In [ ]:
stock_df_dict = {}

# 종목별로 stock_df 만들어서 딕셔너리 stock_df_dict에 저장하기 
for stock_code in tqdm(stock_codes): 

  # 날짜 설정
  dates = pd.date_range('20110104', '20211203', freq = 'B')
  df = pd.DataFrame(dates, columns = ['Date'])

  # 연, 월, 주, 일, 요일, 월별주수 정보 추가
  df['Year'] = df['Date'].dt.year
  df['Month'] = df['Date'].dt.month
  df['Week'] = df['Date'].dt.week
  df['Day'] = df['Date'].dt.day
  df['Weekday'] = df['Date'].dt.weekday
  df['WeekNum'] = np.ceil(df['Day']/7).astype(int)

  # 20110101 ~ 20210910 까지의 주가 정보 가져오기
  stock_value = fdr.DataReader(stock_code, start = '20110104', end = '20211128').reset_index()
  df = pd.merge(df, stock_value, on = 'Date', how = 'left')
  df[stock_value.columns] = df[stock_value.columns].ffill() # 결측치를 이전 값으로 대체 (공휴일 등 쉬는 날에 대하여)
  df = df.dropna() # 남아있는 결측치는 상장전이므로 drop 

  # 종가 데이터 시계열 분해
  df = df.set_index('Date') 
  decomposition = seasonal_decompose(df[:-5]['Close'], model='multiplicative') 
  df['Close_trend'] = decomposition.trend.fillna(method = 'ffill').fillna(method = 'bfill') # 종가 추세 
  df['Close_seosonal'] = decomposition.seasonal.fillna(method = 'ffill').fillna(method = 'bfill') # 종가 계절성 

  # 5일, 20일, 60일, 120일, 240일 이동평균 추가 
  df = add_rolling_mean(df, 'Close', [5, 20, 60, 120, 240]) 

  # lag features 
  col_list = list(df.iloc[:, 6:].columns)
  col_list.remove('Close')
  df = lag_features(df, col_list, 5)
  df.drop(df.index[:5])

  df = df.reset_index()
  df['Index'] = df.index 
  stock_df_dict[stock_code] = df

100%|██████████| 370/370 [03:55<00:00,  1.57it/s]


# 모델링 (XGBoost)

In [ ]:
# SEED 설정 
SEED = 42

# NMAE 함수 정의
def NMAE(true, pred):
 return np.mean(abs(true - pred) / true) * 100 

## public

In [ ]:
# public 
model_oof_score = []
model_pred_dict = {}

for stock_code, df in tqdm(stock_df_dict.items()):
  stock_name = str(stock_list.loc[stock_list['종목코드'] == stock_code]['종목명'].values).strip("['']")
  print(f"\n[종목코드 {stock_code} 종목명 {stock_name} 학습 및 예측]")

  # split train, valid, test (public)
  train = stock_df_dict[stock_code][:-25]
  test = stock_df_dict[stock_code][-25:-20]

  X_train = train[:-5].drop(columns = ['Date', 'Close'])
  y_train = train[:-5]['Close']
  X_valid = train[-5:].drop(columns = ['Date', 'Close'])
  y_valid = train[-5:]['Close']
  X_test = test.drop(columns = ['Date', 'Close'])  

  # xgboost 
  model = ElasticNetCV(random_state = SEED)
  model.fit(X_train, y_train)

  model_oof = model.predict(X_valid)
  model_pred = model.predict(X_test)
  model_pred_dict[stock_code] = model_pred

  # validation NMAE score 
  nmae = NMAE(y_valid, model_oof)
  print(f"\n{train[-5:]['Date'].min().strftime('%Y-%m-%d')} ~ {train[-5:]['Date'].max().strftime('%Y-%m-%d')} 기간동안의 Public OOF NMAE Score : {nmae:.6f}\n")
  model_oof_score.append(nmae)
  print('=' * 120)

print(f'전체 종목에 대한 평균 Public OOF NMAE Score : {np.mean(model_oof_score)}')

  0%|          | 0/370 [00:00<?, ?it/s]


[종목코드 000060 종목명 메리츠화재 학습 및 예측]


  0%|          | 1/370 [00:01<07:50,  1.28s/it]


2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 1.636013


[종목코드 000080 종목명 하이트진로 학습 및 예측]


  1%|          | 2/370 [00:02<07:56,  1.29s/it]

...


[종목코드 352820 종목명 하이브 학습 및 예측]

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 6.834477


[종목코드 357780 종목명 솔브레인 학습 및 예측]


100%|█████████▉| 369/370 [06:24<00:00,  4.69it/s]


2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 2.261219


[종목코드 363280 종목명 티와이홀딩스 학습 및 예측]

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 1.492206


[종목코드 950130 종목명 엑세스바이오 학습 및 예측]


100%|██████████| 370/370 [06:25<00:00,  1.04s/it]


2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 4.535615

전체 종목에 대한 평균 Public OOF NMAE Score : 3.367271940912101


In [ ]:
model_public_sub = pd.DataFrame({'Day' : pd.date_range('20211101', '20211105', freq = 'B')})

for stock_code, pred in model_pred_dict.items():
  stock_pred = pd.DataFrame({stock_code : pred})
  model_public_sub = pd.concat([model_public_sub, stock_pred], axis = 1)

model_public_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,030000,030190,030200,030520,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,29147.0,37774.0,62134.14,151295.0,104622.42,17034.00,46397.5,83346.0,101180.0,711910.0,53364.02,246875.0,34662.30,53542.0,17639.25,2296.20,28082.0,5354.30,346308.59,52567.39,13028.75,34830.5,9304.2,7737.9,31285.20,152890.0,47299.0,87422.0,46170.5,255160.0,288795.0,86876.0,106980.00,33477.0,29152.0,33526.0,157296.12,208325.0,96510.0,98504.0,305040.0,67652.0,33659.0,70251.0,64452.0,13063.0,65232.0,299630.0,42531.5,705100.0,210200.0,9384.15,8860.40,33011.5,512075.0,19130.45,4830.25,91730.0,60377.56,157940.0,111904.0,105057.41,43793.05,30048.5,131620.0,62662.0,560155.0,70252.30,46563.0,105906.0,27555.06,202450.0,238785.0,29626.41,80334.0,182595.0,266995.0,49503.5,96680.0,83928.0,5391.35,78448.47,341260.0,6128.00,23132.5,48528.74,11349.4,311840.0,47428.04,154260.0,15303.0,60680.0,108440.0,79924.0,6863.47,102869.0,11148.0,91966.0,13341.0,35130.0,4753.45,25045.31,122360.0,47326.92,6165.39,35107.5,24699.0,20273.0,31614.5,20931.0,39783.15,178615.0,13038.5,54548.0,20958.00,39258.5,7657.6,14774.5,71191.0,8485.9,8807.42,33273.5,82880.0,23171.53,17861.5,18798.5,257130.0,7091.6,28905.5,401950.0,19354.5,129460.0,185368.0,1882.65,47103.0,10315.8,45330.33,401264.0,6129.27,621420.0,24646.5,45718.28,11156.63,4520.19,99229.0,31476.0,108423.0,40443.0,10080.5,73606.0,10165.20,28214.35,24873.50,29963.5,40590.0,14875.5,6448.9,22530.5,32811.5,131923.0,25441.0,19756.96,22806.5,43276.5,1373080.0,

## private

In [ ]:
# private
lr_oof_score = []
lr_pred_dict = {}

for stock_code, df in tqdm(stock_df_dict.items()):
  stock_name = str(stock_list.loc[stock_list['종목코드'] == stock_code]['종목명'].values).strip("['']")
  print(f"\n[종목코드 {stock_code} 종목명 {stock_name} 학습 및 예측]")

  # split train, valid, test (private)
  train = stock_df_dict[stock_code][:-5]
  test = stock_df_dict[stock_code][-5:]

  X_train = train[:-5].drop(columns = ['Date', 'Close'])
  y_train = train[:-5]['Close']
  X_valid = train[-5:].drop(columns = ['Date', 'Close'])
  y_valid = train[-5:]['Close']
  X_test = test.drop(columns = ['Date', 'Close'])  

  # xgboost 
  model = ElasticNetCV(random_state = SEED)
  model.fit(X_train, y_train)

  model_oof = model.predict(X_valid)
  model_pred = model.predict(X_test)
  model_pred_dict[stock_code] = model_pred

  # validation NMAE score 
  nmae = NMAE(y_valid, model_oof)
  print(f"\n{train[-5:]['Date'].min().strftime('%Y-%m-%d')} ~ {train[-5:]['Date'].max().strftime('%Y-%m-%d')} 기간동안의 Public OOF NMAE Score : {nmae:.6f}\n")
  model_oof_score.append(nmae)
  print('=' * 120)

print(f'전체 종목에 대한 평균 Private OOF NMAE Score : {np.mean(model_oof_score)}')

  0%|          | 0/370 [00:00<?, ?it/s]


[종목코드 000060 종목명 메리츠화재 학습 및 예측]


  0%|          | 1/370 [00:01<07:40,  1.25s/it]


2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 4.670870


[종목코드 000080 종목명 하이트진로 학습 및 예측]


  1%|          | 2/370 [00:02<07:49,  1.28s/it]

...


2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 10.449228


[종목코드 363280 종목명 티와이홀딩스 학습 및 예측]


100%|█████████▉| 369/370 [06:27<00:00,  4.60it/s]


2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 1.722842


[종목코드 950130 종목명 엑세스바이오 학습 및 예측]


100%|██████████| 370/370 [06:28<00:00,  1.05s/it]


2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 16.091951

전체 종목에 대한 평균 Private OOF NMAE Score : 3.804783249479059


In [ ]:
model_private_sub = pd.DataFrame({'Day' : pd.date_range('20211129', '20211203', freq = 'B')})

for stock_code, pred in model_pred_dict.items():
  stock_pred = pd.DataFrame({stock_code : pred})
  model_private_sub = pd.concat([model_private_sub, stock_pred], axis = 1)

model_private_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,030000,030190,030200,030520,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-29,28309.0,32812.0,60662.66,136455.0,136551.54,16353.90,46892.0,82515.0,110140.0,694400.0,49985.02,218465.0,31988.83,67234.0,16434.18,2098.70,25462.0,5317.35,333682.70,46588.27,13144.37,33386.0,10306.4,7861.9,30041.78,145160.0,43296.5,80240.0,40134.5,233545.0,290780.0,84737.0,96238.06,32989.5,25202.5,33939.5,139997.27,209655.0,98617.0,100647.0,284005.0,55255.0,31895.0,74224.0,66878.0,13057.5,53356.0,237545.0,42591.0,755160.0,180465.0,9055.09,9021.35,32789.5,489760.0,19023.18,5050.05,77382.0,56464.58,173980.0,85796.0,101806.09,35091.54,28369.0,117047.0,57538.0,502015.0,72627.42,44870.5,88757.0,25890.75,281125.0,209415.0,26670.22,76141.0,163430.0,240380.0,42117.5,81755.0,81818.0,5092.20,90962.52,316970.0,6088.18,22821.5,48435.24,11174.2,310025.0,44582.23,153680.0,14091.5,48591.5,125630.0,76588.0,6239.62,100204.0,11054.0,105890.0,11703.8,36933.0,4400.25,22298.44,112165.0,40656.75,5386.66,34038.5,23366.0,19856.5,30759.5,23751.0,39214.76,153030.0,14372.0,49392.0,18523.03,39863.5,7759.6,13716.0,65581.0,8384.5,11266.71,34161.5,83806.0,24027.38,22862.0,16003.0,252205.0,5868.0,25492.0,402720.0,17844.0,126315.0,171336.0,1764.70,53433.0,7834.1,38515.18,385597.0,7195.29,761710.0,24979.5,41781.20,14936.35,4896.85,114168.0,30618.0,109516.0,35066.0,9867.9,80783.0,8248.88,26523.93,26077.0,34651.0,35894.0,15694.5,5950.7,19867.0,29552.5,120494.0,21917.0,20377.69,23166.0,42471.5,1182220.0,65

# Submission

In [ ]:
model_sub = pd.concat([model_public_sub, model_private_sub], ignore_index = True)
model_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,030000,030190,030200,030520,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,29147.0,37774.0,62134.14,151295.0,104622.42,17034.00,46397.5,83346.0,101180.0,711910.0,53364.02,246875.0,34662.30,53542.0,17639.25,2296.20,28082.0,5354.30,346308.59,52567.39,13028.75,34830.5,9304.2,7737.9,31285.20,152890.0,47299.0,87422.0,46170.5,255160.0,288795.0,86876.0,106980.00,33477.0,29152.0,33526.0,157296.12,208325.0,96510.0,98504.0,305040.0,67652.0,33659.0,70251.0,64452.0,13063.0,65232.0,299630.0,42531.5,705100.0,210200.0,9384.15,8860.40,33011.5,512075.0,19130.45,4830.25,91730.0,60377.56,157940.0,111904.0,105057.41,43793.05,30048.5,131620.0,62662.0,560155.0,70252.30,46563.0,105906.0,27555.06,202450.0,238785.0,29626.41,80334.0,182595.0,266995.0,49503.5,96680.0,83928.0,5391.35,78448.47,341260.0,6128.00,23132.5,48528.74,11349.4,311840.0,47428.04,154260.0,15303.0,60680.0,108440.0,79924.0,6863.47,102869.0,11148.0,91966.0,13341.0,35130.0,4753.45,25045.31,122360.0,47326.92,6165.39,35107.5,24699.0,20273.0,31614.5,20931.0,39783.15,178615.0,13038.5,54548.0,20958.00,39258.5,7657.6,14774.5,71191.0,8485.9,8807.42,33273.5,82880.0,23171.53,17861.5,18798.5,257130.0,7091.6,28905.5,401950.0,19354.5,129460.0,185368.0,1882.65,47103.0,10315.8,45330.33,401264.0,6129.27,621420.0,24646.5,45718.28,11156.63,4520.19,99229.0,31476.0,108423.0,40443.0,10080.5,73606.0,10165.20,28214.35,24873.50,29963.5,40590.0,14875.5,6448.9,22530.5,32811.5,131923.0,25441.0,19756.96,22806.5,43276.5,1373080.0,

In [ ]:
model_sub.to_csv(path + 'ElasticNetCV.csv', index = False)